## Let's try to compare the mmd of the 2 distributions to one nyström subsampled bucked for X and one not subsampled bucket for Y

In [203]:
from mmdew.bucket_stream2 import BucketStream
from mmdew.bucket_stream_old import BucketStream as OldBucketStream
import numpy as np
import pandas as pd
from mmdew.mmd import MMD
mymmd = MMD(biased=True, gamma=1)
from sklearn import metrics
import numpy.linalg as la
np.random.seed(40)
X = np.random.normal(0, 1, 32)
Y = np.random.normal(1, 2, 32)


m_idx = np.random.default_rng().integers(32, size=5)
subsample = X[m_idx]
start_weights = ((1/32) * la.pinv(metrics.pairwise.linear_kernel(subsample.reshape(-1,1),subsample.reshape(-1,1)))
                 @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1),X.reshape(-1,1)) @ np.ones(32))



end_weights = np.ones((32,1))/32
addend_1 = start_weights.T @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1), subsample.reshape(-1,1)) @ start_weights
addend_2 = end_weights.T @ metrics.pairwise.linear_kernel(Y.reshape(-1,1), Y.reshape(-1,1)) @ end_weights
addend_3 = start_weights.T @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1), Y.reshape(-1,1)) @ end_weights
print(f"real mmd:{mymmd.mmd(X.reshape(-1,1),Y.reshape(-1,1))}")
print(f"nyström mmd: {(addend_1 + addend_2 - (2 * addend_3))[0][0]}")



real mmd:0.6016100063216595
nyström mmd: 0.6016100063216595


## MMD is identical
now let's try to create 2 subsampled buckets from X, merge them into a third bucket with the elements: new_subsample and the weights: new_weights, then check MMD

In [204]:
import math

X_1 = np.random.normal(0, 1, 32)
X_2 = np.random.normal(0, 1, 32)
Y = np.random.normal(1, 2, 64)


m_idx1 = np.random.default_rng().integers(32, size=5)
subsample1 = X_1[m_idx1]
m_idx2 = np.random.default_rng().integers(32, size=5)
subsample2 = X_2[m_idx1]
bucket1_weights = ((1/32) * la.pinv(metrics.pairwise.linear_kernel(subsample1.reshape(-1,1),subsample1.reshape(-1,1)))
                 @ metrics.pairwise.linear_kernel(subsample1.reshape(-1,1),X_1.reshape(-1,1)) @ np.ones(32))
bucket2_weights = ((1/32) * la.pinv(metrics.pairwise.linear_kernel(subsample2.reshape(-1,1),subsample2.reshape(-1,1)))
                 @ metrics.pairwise.linear_kernel(subsample2.reshape(-1,1),X_2.reshape(-1,1)) @ np.ones(32))

#getting new elements
joined_subsamples = np.concatenate((subsample1, subsample2))
joined_uncompressed_capacity = 64
m = round(math.sqrt(64))
m_idx_new = np.random.default_rng().integers(len(joined_subsamples), size=m)
new_subsample = joined_subsamples[m_idx_new]

#getting new weights
new_subsample = new_subsample.reshape(-1,1)
joined_subsamples = joined_subsamples.reshape(-1,1)
joined_weights = np.concatenate((bucket1_weights,bucket2_weights))
K_z = metrics.pairwise.linear_kernel(new_subsample, joined_subsamples)
K_m_inv = la.pinv(metrics.pairwise.linear_kernel(new_subsample, new_subsample))
new_weights = .5 * K_m_inv @ K_z @ joined_weights

#weights for Y
end_weights = np.ones((64,1))/64

addend_1 = new_weights.T @ metrics.pairwise.linear_kernel(new_subsample.reshape(-1,1), new_subsample.reshape(-1,1)) @ new_weights
addend_2 = end_weights.T @ metrics.pairwise.linear_kernel(Y.reshape(-1,1), Y.reshape(-1,1)) @ end_weights
addend_3 = new_weights.T @ metrics.pairwise.linear_kernel(new_subsample.reshape(-1,1), Y.reshape(-1,1)) @ end_weights

print(f"real mmd:{mymmd.mmd(np.concatenate((X_1,X_2)).reshape(-1,1),Y.reshape(-1,1))}")
print(f"nyström mmd of bucket, made of subsampled buckets: {(addend_1 + addend_2 - (2 * addend_3))[0][0]}")
breakpoint()




real mmd:2.3226094951499947
nyström mmd of bucket, made of subsampled buckets: 2.3226094951499943


## MMD is equal 
why does this not work in bucketstream2.py ? 

In [205]:
from mmdew.bucket_stream2 import BucketStream
from mmdew.bucket_stream_old import BucketStream as OldBucketStream
import numpy as np
import pandas as pd
from mmdew.mmd import MMD
mymmd = MMD(biased=True, gamma=1)
if __name__ == "__main__":
    bs_ss = BucketStream(gamma=1)
    bs_no_ss = BucketStream(gamma=1, apply_subsampling=False)
    bs_old = OldBucketStream(gamma=1)

    errors = []

    for exponent in range(1, 11):
        iter_errors = []
        for iteration in range(1,11):

            bs_ss = BucketStream(gamma=1)
            bs_no_ss = BucketStream(gamma=1, apply_subsampling=False)
            bs_old = OldBucketStream(gamma=1)
            limit = 2 ** exponent
            X = np.random.normal(0, 1, limit)
            Y = np.random.normal(1, 2, limit - 1)
           
            for i in range(0, limit):
                bs_ss.insert_no_cut(X[i])
             
            for i in range(0, limit - 1):
                bs_ss.insert_no_cut(Y[i])
      
            ss_mmd = bs_ss.mmd(1)[0]
            real_mmd = mymmd.mmd(X.reshape(-1, 1),Y.reshape(-1, 1))
            #print(f"SS:    exponent: {exponent} bucket count: {len(bs_ss.buckets)} mmd value={ss_mmd}")
         
            #print(f"real MMD: {real_mmd}")
            iter_errors = np.concatenate(( iter_errors, [(abs(real_mmd - ss_mmd))]))
            #if iteration == 1 and exponent == 5 :
            #    breakpoint()
        errors = np.concatenate((errors, [np.array(iter_errors).mean()] ))

    for i in range(0,10):
        print(f"error for exponent {i+1} is {errors[i]}")


error for exponent 1 is 6.938893903907228e-16
error for exponent 2 is 1.2212453270876722e-15
error for exponent 3 is 1.2212453270876722e-15
error for exponent 4 is 6.161737786669619e-16
error for exponent 5 is 7.93809462606987e-16
error for exponent 6 is 0.27322697803554885
error for exponent 7 is 0.29218514919762306
error for exponent 8 is 0.2324963532574406
error for exponent 9 is 0.2465660856119707
error for exponent 10 is 0.1989622980445256


# Exponent 6 is the first time that we subsample with depth > 1 
But we just showed that this isn't the problem. Let's have a deeper look

In [206]:
bs_ss = BucketStream(gamma=1)
exponent = 6
limit = 2 ** exponent
X = np.random.normal(0, 1, limit)
Y = np.random.normal(1, 2, limit - 1)

for i in range(0, limit):
    bs_ss.insert_no_cut(X[i])
 
for i in range(0, limit - 1):
    bs_ss.insert_no_cut(Y[i])

ss_mmd = bs_ss.mmd(1)[0]
real_mmd = mymmd.mmd(X.reshape(-1, 1),Y.reshape(-1, 1))
print(f"SS:    exponent: {exponent} bucket count: {len(bs_ss.buckets)} mmd value={ss_mmd}")
print(f"real MMD: {real_mmd}\n")
for bucket in bs_ss.buckets:
    print(f"Element count: {len(bucket.elements)}" )
    print(bucket)
    print("\n")



SS:    exponent: 6 bucket count: 7 mmd value=0.3138935548362058
real MMD: 0.16034832195908985

Element count: 8
Elems:	[[-0.61609615]
 [-0.58468393]
 [-0.60527542]
 [ 1.30946385]
 [ 0.46623726]
 [-0.11185297]
 [-0.50221412]
 [-0.90660695]]
Weights:	[[-0.00683258]
 [-0.00648422]
 [-0.00671258]
 [ 0.01452212]
 [ 0.00517063]
 [-0.00124046]
 [-0.00556962]
 [-0.01005438]]
Uncomp_Cap:	64


Element count: 6
Elems:	[[-5.16731105]
 [ 1.442592  ]
 [ 0.80318161]
 [-1.25148157]
 [-0.55198791]
 [ 1.65023783]]
Weights:	[[-0.03319136]
 [ 0.00926625]
 [ 0.0051591 ]
 [-0.00803868]
 [-0.0035456 ]
 [ 0.01060003]]
Uncomp_Cap:	32


Element count: 16
Elems:	[[ 3.96456395]
 [-3.91575867]
 [ 5.60934544]
 [-0.15462216]
 [ 4.089723  ]
 [-2.63150713]
 [ 0.51620667]
 [ 0.76058081]
 [ 0.70668094]
 [ 0.30310334]
 [ 1.38158431]
 [-1.10191109]
 [-0.56608032]
 [-0.83511113]
 [ 2.38796535]
 [ 1.58207457]]
Weights:	[[ 0.02998238]
 [-0.02961328]
 [ 0.04242119]
 [-0.00116934]
 [ 0.0309289 ]
 [-0.01990101]
 [ 0.00390386]
 

## This does not look to bad. (Except for the mmd disparity of course)
The amount of subsamples is right. the weight count matches...
Let's try to reconstruct the mmd calc

In [207]:
start = bs_ss.buckets[:1]
end = bs_ss.buckets[1:]
start_elements = start[0].elements
start_weights = start[0].weights * len(start[0].weights)
end_elements = end[0].elements
end_weights = end[0].weights * len(end[0].weights)
start_uncompressed_capacity = start[0].uncompressed_capacity
end_uncompressed_capacity = end[0].uncompressed_capacity
for bucket in start[1:]:
    #breakpoint()
    start_elements = np.concatenate((start_elements, bucket.elements))
    start_weights = np.concatenate((start_weights, bucket.weights * len(bucket.weights)))
    start_uncompressed_capacity += bucket.uncompressed_capacity
for bucket in end[1:]:
    #breakpoint()
    end_elements = np.concatenate((end_elements, bucket.elements))
    #breakpoint()
    end_weights = np.concatenate((end_weights, bucket.weights * len(bucket.weights)))
    end_uncompressed_capacity += bucket.uncompressed_capacity
start_capacity = len(start_elements)
end_capacity = len(end_elements)
print(start_uncompressed_capacity)
print(start_capacity)
print(end_uncompressed_capacity)
print(end_capacity)

64
8
63
37


## Yes, everything seems to be alright
37 is exactly the amount of samples that should be right of the split because of: 31 elements from the buckets without subsampling and 5 (+1 for bad rounding) from the subsampled u-cap 32 bucket
## Let's look deeper...

In [208]:
start_weights = start_weights * (1/start_capacity)
end_weights = end_weights * (1/end_capacity)
#breakpoint()
print("Start Weights")
print(start_weights)
print("\nEnd Weights")
print(end_weights)

Start Weights
[[-0.00683258]
 [-0.00648422]
 [-0.00671258]
 [ 0.01452212]
 [ 0.00517063]
 [-0.00124046]
 [-0.00556962]
 [-0.01005438]]

End Weights
[[-5.38238211e-03]
 [ 1.50263479e-03]
 [ 8.36611207e-04]
 [-1.30357007e-03]
 [-5.74962452e-04]
 [ 1.71892314e-03]
 [ 1.29653518e-02]
 [-1.28057434e-02]
 [ 1.83442965e-02]
 [-5.05662354e-04]
 [ 1.33746606e-02]
 [-8.60584316e-03]
 [ 1.68815566e-03]
 [ 2.48733477e-03]
 [ 2.31106551e-03]
 [ 9.91241769e-04]
 [ 4.51820852e-03]
 [-3.60359049e-03]
 [-1.85125793e-03]
 [-2.73107198e-03]
 [ 7.80938613e-03]
 [ 5.17387376e-03]
 [-3.80190578e-04]
 [-2.78504311e-03]
 [ 1.45997206e-02]
 [-3.01481298e-03]
 [ 2.08668325e-02]
 [ 8.66238811e-03]
 [-8.14304931e-03]
 [ 5.00234369e-05]
 [-7.58531371e-03]
 [-5.79207111e-04]
 [ 5.76552872e-03]
 [ 8.34982908e-03]
 [ 3.17980058e-02]
 [ 1.90729591e-02]
 [ 2.70270270e-02]]


In [209]:
addend_1 = start_weights.T @ metrics.pairwise.linear_kernel(start_elements, start_elements) @ start_weights
addend_2 = end_weights.T @ metrics.pairwise.linear_kernel(end_elements, end_elements) @ end_weights
addend_3 = start_weights.T @ metrics.pairwise.linear_kernel(start_elements, end_elements) @ end_weights
print(addend_1)
print(addend_2)
print(addend_3)
print(addend_1+addend_2-(2*addend_3))

[[0.00207406]]
[[0.36699842]]
[[0.02758946]]
[[0.31389355]]


Ok let's start over

In [210]:
from mmdew.bucket_stream2 import Bucket

def k( x, y):

    #return metrics.pairwise.rbf_kernel(x,y, gamma=self.gamma)
    return metrics.pairwise.linear_kernel(x,y)


def mmd(split, buckets):
    start = buckets[:split]
    end = buckets[split:]
    #breakpoint()
    start_elements = start[0].elements
    start_weights = start[0].weights * start[0].uncompressed_capacity
    end_elements = end[0].elements
    end_weights = end[0].weights * end[0].uncompressed_capacity
    start_uncompressed_capacity = start[0].uncompressed_capacity
    end_uncompressed_capacity = end[0].uncompressed_capacity
    #breakpoint()
    for bucket in start[1:]:
        #breakpoint()
        start_elements = np.concatenate((start_elements, bucket.elements))
        start_weights = np.concatenate((start_weights, bucket.weights * bucket.uncompressed_capacity))
        start_uncompressed_capacity += bucket.uncompressed_capacity
    for bucket in end[1:]:
        #breakpoint()
        end_elements = np.concatenate((end_elements, bucket.elements))
        #breakpoint()
        end_weights = np.concatenate((end_weights, bucket.weights * bucket.uncompressed_capacity))
        end_uncompressed_capacity += bucket.uncompressed_capacity
    #

    start_capacity = len(start_elements)
    end_capacity = len(end_elements)
    start_weights = start_weights * (1/start_uncompressed_capacity)
    end_weights = end_weights * (1/end_uncompressed_capacity)
    #breakpoint()
    addend_1 = start_weights.T @ k(start_elements, start_elements) @ start_weights
    addend_2 = end_weights.T @ k(end_elements, end_elements) @ end_weights
    addend_3 = start_weights.T @ k(start_elements, end_elements) @ end_weights
    return (addend_1 + addend_2 - (2 * addend_3))[0][0], start_uncompressed_capacity, end_uncompressed_capacity
def merge_buckets_with_subsampling(bucket_list):
    """Merges the buckets in `bucket_list` such that one bucket remains with XX, and XY such that their values correspond to the case that all data would have been in this bucket."""
    if len(bucket_list) == 1:
        return bucket_list[0]
    current = bucket_list[-1]
    previous = bucket_list[-2]

    current_elements = current.elements
    previous_elements = previous.elements
    current_weights = current.weights
    previous_weights = previous.weights

    joined_elements = np.concatenate((current_elements, previous_elements))
    joined_uncompressed_capacity = current.uncompressed_capacity + previous.uncompressed_capacity
    #subsampling seems to be too extreme. Maybe select less aggressively
    #maybe choose combined uncompressed capacity as n which would probably not contradict the chatalic paper
    #breakpoint()
    if joined_uncompressed_capacity > 16:

        m = round(math.sqrt(joined_uncompressed_capacity))  # size of the subsample
        #ToDo: uncomment this
        #m_idx = np.random.default_rng().integers(len(joined_elements), size=m)
        m_idx = range(0,m)
        subsample = joined_elements[m_idx]
    else:
        m = joined_uncompressed_capacity
        subsample = joined_elements
   # assuming current_elements and previous_elements have the same length


    joined_weights = np.concatenate((current_weights, previous_weights))
    #breakpoint()
    K_z = k(subsample, joined_elements)



    #K_m = np.zeros((m, m))  # initialize the kernel matrix with zeros
    #for i in range(m):
        #for j in range(m):
            # reshape to 2D array as rbf_kernel expects 2D array

    K_m = k(subsample, subsample)
    K_m_inv = la.pinv(K_m)
    #breakpoint()
    new_weights = .5 * K_m_inv @ K_z @ joined_weights
    return merge_buckets_with_subsampling(
        bucket_list[:-2]
        + [
            Bucket(
                elements=subsample,
                weights=new_weights,
                capacity=m,
                uncompressed_capacity=joined_uncompressed_capacity
            )
        ]
    )


    #print(f"split: {split} start_uncompressed_capacity: {start_uncompressed_capacity} und end_uncompressed_capacity: {end_uncompressed_capacity}")
    


In [211]:
np.random.seed(40)
X = np.random.normal(0, 1, 128)
Y = np.random.normal(1, 2, 127)

In [212]:
bs = []
def insert_no_cut(buckets, element):
    #breakpoint()
    buckets += [
        Bucket(
            elements=np.array(element).reshape(1,-1),
            weights=np.array(1).reshape(1,-1),
            capacity=1,
            uncompressed_capacity=1
        )
    ]
    return merge(buckets)
    
    #breakpoint()
    
def merge(buckets):
    if len(buckets) < 2:
        return buckets
    current = buckets[-1]
    previous = buckets[-2]
    if previous.uncompressed_capacity == current.uncompressed_capacity:
        buckets = buckets[:-2] + [merge_buckets_with_subsampling(buckets[-2:])]
        buckets = merge(buckets)
    return buckets

for i in range(0,128):
    bs = insert_no_cut(bs, X[i])
for i in range(0,127):
    bs = insert_no_cut(bs, Y[i])
    


In [213]:
print(bs[1])

Elems:	[[ 3.00111581]
 [ 4.52203497]
 [-1.25229966]
 [ 0.21688528]
 [ 2.59023479]
 [ 2.75207267]
 [ 0.62721281]
 [-0.48475151]]
Weights:	[[ 0.08257456]
 [ 0.12442208]
 [-0.03445655]
 [ 0.00596752]
 [ 0.07126933]
 [ 0.07572224]
 [ 0.01725752]
 [-0.01333775]]
Uncomp_Cap:	64


In [214]:
mmd(1,bs)

(1.1663497414336537, 128, 127)

In [215]:
mymmd.mmd(X.reshape(-1,1),Y.reshape(-1,1))

1.166349741433654